In [1]:
# Giovani Nascimento Pereira - 168609
# Carlos Augusto Figueiredo Feire de Carvalho - 165684

import numpy
import sys
from PIL import Image, ImageFilter
from numpy import genfromtxt
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA
import timeit
import csv
import kmedoids
import numpy
import sys
from PIL import Image, ImageFilter
from numpy import genfromtxt
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_samples, silhouette_score
import csv

from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import matplotlib.cm as cm

nTotal = 19924
nTotal = 2000
nFeatures = 2209
nTreino = nTotal

#Atualizar com o caminho do csv no seu diretorio
csvPath = "./dataset/csv.noUp/data.csv"


# Pegando os dados
print('-----------Pegando os dados----------')
reader = csv.reader(open(csvPath, "r"), delimiter=",")
med = list(reader)
data= numpy.array(med).astype("float")

# print(data.shape)
# Dimensao dos dados completos: (19924, 2209) -> as expected
# 19924 documentos
# 2209 features

# Separa em treino
print('-----------Montando Treino----------')
X_treino = numpy.ones((nTreino, nFeatures + 1))
for num in range(0, nTreino):
	X_treino[num, 1:(nFeatures+1)] = data[num, 0:nFeatures]


-----------Pegando os dados----------
-----------Montando Treino----------


In [28]:
# Usando o metodo do elbow
# Reference1: https://pythonprogramminglanguage.com/kmeans-elbow-method/
# Reference2: http://www.awesomestats.in/python-cluster-validation/

print('------ Definindo melhor numero de clusters pelo elbow ----')
kmeans = KMeans(random_state = 1)
distortions = []

X = X_treino

# k means determine k
distortions1 = []
distortions2 = []
K = range(10, 150)
for k in K:
    print(k , ' ')
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    
    # Reference 1
    distortions1.append(sum(numpy.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
     
    # Reference 2
    distortions2.append(kmeanModel.inertia_)

plt.plot(K, distortions1, 'o-')
plt.xlabel('Numero de Clusters - euclidean coisa')
plt.ylabel('Erro')
plt.title('Reference 1')
plt.show()

plt.plot(K, distortions2, 'o-')
plt.xlabel('Numero de Clusters')
plt.ylabel('Erro')
plt.title('Reference 2 - .inertia_')
plt.show()

print('------> Done')

------ Definindo melhor numero de clusters pelo elbow ----
10  
11  


KeyboardInterrupt: 

In [ ]:
# Fazendo pelo Silhouette de novo
# Reference: http://www.awesomestats.in/python-cluster-validation/

print('-------- Definindo melhor numero de clusters pelo Silhouette --------')

X_scaled = X_treino
silhuetas_medias = []

cluster_range = range( 10, 120 )

for n_clusters in cluster_range:
  # Create a subplot with 1 row and 2 columns
  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.set_size_inches(18, 7)

  # The 1st subplot is the silhouette plot
  # The silhouette coefficient can range from -1, 1 but in this example all
  # lie within [-0.1, 1]
  ax1.set_xlim([-0.1, 1])
  # The (n_clusters+1)*10 is for inserting blank space between silhouette
  # plots of individual clusters, to demarcate them clearly.
  ax1.set_ylim([0, len(X_scaled) + (n_clusters + 1) * 10])

  # Initialize the clusterer with n_clusters value and a random generator
  # seed of 10 for reproducibility.
  clusterer = KMeans(n_clusters=n_clusters, random_state=10)
  cluster_labels = clusterer.fit_predict( X_scaled )

  # The silhouette_score gives the average value for all the samples.
  # This gives a perspective into the density and separation of the formed
  # clusters
  silhouette_avg = silhouette_score(X_scaled, cluster_labels, sample_size = 1000)
  print("For n_clusters =", n_clusters,
        "The average silhouette_score is :", silhouette_avg)
  silhuetas_medias.append(silhouette_avg)

  # Compute the silhouette scores for each sample
  sample_silhouette_values = silhouette_samples(X_scaled, cluster_labels)

  y_lower = 10
  for i in range(n_clusters):
      # Aggregate the silhouette scores for samples belonging to
      # cluster i, and sort them
      ith_cluster_silhouette_values = \
          sample_silhouette_values[cluster_labels == i]

      ith_cluster_silhouette_values.sort()

      size_cluster_i = ith_cluster_silhouette_values.shape[0]
      y_upper = y_lower + size_cluster_i

      color = cm.spectral(float(i) / n_clusters)
      ax1.fill_betweenx(numpy.arange(y_lower, y_upper),
                        0, ith_cluster_silhouette_values,
                        facecolor=color, edgecolor=color, alpha=0.7)

      # Label the silhouette plots with their cluster numbers at the middle
      ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

      # Compute the new y_lower for next plot
      y_lower = y_upper + 10  # 10 for the 0 samples

  ax1.set_title("The silhouette plot for the various clusters.")
  ax1.set_xlabel("The silhouette coefficient values")
  ax1.set_ylabel("Cluster label")

  # The vertical line for average silhoutte score of all the values
  ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

  ax1.set_yticks([])  # Clear the yaxis labels / ticks
  ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

  # 2nd Plot showing the actual clusters formed
  colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
  ax2.scatter(X_scaled[:, 0], X_scaled[:, 1], marker='.', s=30, lw=0, alpha=0.7,
              c=colors)

  # Labeling the clusters
  centers = clusterer.cluster_centers_
  # Draw white circles at cluster centers
  ax2.scatter(centers[:, 0], centers[:, 1],
              marker='o', c="white", alpha=1, s=200)

  for i, c in enumerate(centers):
      ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

  ax2.set_title("The visualization of the clustered data.")
  ax2.set_xlabel("Feature space for the 1st feature")
  ax2.set_ylabel("Feature space for the 2nd feature")

  plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                "with n_clusters = %d" % n_clusters),
               fontsize=14, fontweight='bold')

  plt.show()

print('-----> Done')

In [2]:
#  Segundo teste, pegamos o melhor resultado do primeiro cluster e mudamos
# o init, que por padrão era kMeans++, p/ random
#

kmeans2 = KMeans(random_state = 1, init = 'random', n_clusters = 78)
#timeit.timeit('kmeans2.fit(X_treino)')
label_teste = kmeans2.fit_predict(X_treino)
print('Mudando o método de inicialização do centroides para random: ')
silhouette_score_new = silhouette_score(X_treino, label_teste)
print(silhouette_score_new, '\n')

# Fim do segundo teste
#
#

Mudando o método de inicialização do centroides para random: 
0.0541362906245 



In [3]:
# Fazer um teste com mini-batch k-means
#
#
miniKmeans = MiniBatchKMeans(n_clusters = 78)
label_mini = miniKmeans.fit_predict(X_treino)
print('Mudando para MiniBatchKMeans temos: ', silhouette_score(X_treino, label_mini), '\n')

Mudando para MiniBatchKMeans temos:  0.00509597735441 



In [4]:
# MEDOIDS
# https://github.com/salspaugh/machine_learning/blob/master/clustering/kmedoids.py

distancia_treino = pairwise_distances(X_treino, metric='euclidean')
clusters, medoids = kmedoids.cluster(distancia_treino, 78)
print('Numero de clusters: ', 78)
print('Mudando para K-Medoids temos: ', silhouette_score(X_treino, clusters), '\n')

#
#

Numero de clusters:  78
Mudando para K-Medoids temos:  0.0389950275468 



In [5]:
#	APLICANDO PCA
#
#
for i in range (1, 11):
    pca = PCA(n_components= i/10, svd_solver='full')
    X_treino_pca = pca.fit_transform(X_treino)
    for j in range(1, 10):
        kmeans_pca = KMeans(n_clusters=10 * (j - 1) + 2)
        label_teste = kmeans_pca.fit_predict(X_treino_pca)

        print('Aplicando PCA no nosso melhor modelo com clusters:', 10*(j - 1) + 2, 'e variancia: ', i/10)
        # calcula e imprime os silhouette_score desse modelo com esse numero de clusters
        silhouette_score_new = silhouette_score(X_treino_pca, label_teste)
        print(silhouette_score_new, '\n')

#
#
#

KeyboardInterrupt: 